In [ ]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Comment this if the data visualisations doesn't work on your side
%matplotlib inline

: 

In [ ]:
print("TensorFlow v" + tf.__version__)
print("TensorFlow Decision Forests v" + tfdf.__version__)

In [ ]:
train_file_path = "D:\House_price_prediction\train.csv"
dataset_df = pd.read_csv(train_file_path)
print("Full train dataset shape is {}".format(dataset_df.shape))

In [ ]:
dataset_df.head(3)

In [ ]:
dataset_df = dataset_df.drop('srp', axis=1)
dataset_df.head(3)

In [ ]:
dataset_df.info()

In [ ]:
# Apply log transformation to price_info, adding a small value to avoid log(0)
dataset_df['log_price_info'] = np.log1p(dataset_df['price_info'])

# Display descriptive statistics for log-transformed price_info
print(dataset_df['log_price_info'].describe())

# Plot the distribution of the log-transformed price_info
plt.figure(figsize=(9, 8))
sns.histplot(dataset_df['log_price_info'], color='g', bins=100, kde=True, stat="density")
plt.title('Log-Transformed Price Info Distribution')
plt.xlabel('Log of Price Info')
plt.ylabel('Density')
plt.show()

In [ ]:
list(set(dataset_df.dtypes.tolist()))

In [ ]:
df_num = dataset_df.select_dtypes(include = ['float64', 'int64'])
df_num.head()

In [ ]:
df_num.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8);

In [ ]:
import numpy as np

def split_dataset(dataset, test_ratio=0.30):
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

train_ds_pd, valid_ds_pd = split_dataset(dataset_df)
print("{} examples in training, {} examples in testing.".format(
    len(train_ds_pd), len(valid_ds_pd)))

In [ ]:
label = 'price-info'
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label, task = tfdf.keras.Task.REGRESSION)
valid_ds = tfdf.keras.pd_dataframe_to_tf_dataset(valid_ds_pd, label=label, task = tfdf.keras.Task.REGRESSION)

In [ ]:
tfdf.keras.get_all_models()

In [ ]:
rf = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION)
rf.compile(metrics=["mse"]) # Optional, you can use this to include a list of eval metrics

In [ ]:
rf.fit(x=train_ds)

In [ ]:
import matplotlib.pyplot as plt
logs = rf.make_inspector().training_logs()
plt.plot([log.num_trees for log in logs], [log.evaluation.rmse for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("RMSE (out-of-bag)")
plt.show()

In [ ]:
inspector = rf.make_inspector()
inspector.evaluation()

In [ ]:
print(f"Available variable importances:")
for importance in inspector.variable_importances().keys():
  print("\t", importance)

In [ ]:
inspector.variable_importances()["NUM_AS_ROOT"]

In [ ]:
plt.figure(figsize=(12, 4))

# Mean decrease in AUC of the class 1 vs the others.
variable_importance_metric = "NUM_AS_ROOT"
variable_importances = inspector.variable_importances()[variable_importance_metric]

# Extract the feature name and importance values.
#
# `variable_importances` is a list of <feature, importance> tuples.
feature_names = [vi[0].name for vi in variable_importances]
feature_importances = [vi[1] for vi in variable_importances]
# The feature are ordered in decreasing importance value.
feature_ranks = range(len(feature_names))

bar = plt.barh(feature_ranks, feature_importances, label=[str(x) for x in feature_ranks])
plt.yticks(feature_ranks, feature_names)
plt.gca().invert_yaxis()

# TODO: Replace with "plt.bar_label()" when available.
# Label each bar with values
for importance, patch in zip(feature_importances, bar.patches):
  plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{importance:.4f}", va="top")

plt.xlabel(variable_importance_metric)
plt.title("NUM AS ROOT of the class 1 vs the others")
plt.tight_layout()
plt.show()

In [ ]:
# Load the test data
test_file_path = "/kaggle/input/house-test-data/test.csv"
test_data = pd.read_csv(test_file_path)

# Extract the 'srp' column and remove it from the test data
ids = test_data.pop('srp')

# Ensure 'log_price_info' is included in the test data
def add_log_price_info(df):
    df['log_price_info'] = np.log1p(df['price_info'])  # Use log1p for numerical stability
    return df

# Add the log_price_info feature
test_data = add_log_price_info(test_data)

# Convert the DataFrame to a TensorFlow dataset for prediction
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_data, task=tfdf.keras.Task.REGRESSION)

# Ensure the model 'rf' is defined and loaded correctly
# rf = tfdf.keras.RandomForestModel(task=tfdf.keras.Task.REGRESSION)

# Predict using the RF model
preds = rf.predict(test_ds)

# Convert predictions to a DataFrame
output = pd.DataFrame({'srp': ids, 'price-info': preds.squeeze()})

# Display the first few rows of the output
print(output.head())